In [1]:
import math
import itertools
import pandas
import numpy
from scipy.spatial import distance
from sklearn.metrics.pairwise import cosine_similarity
from scipy.stats import pearsonr

In [2]:
pandas.set_option('display.max_rows', None)

In [3]:
enum = { "beginner": 0, "intermediate": 1, "advanced": 2 }
inverse_enum = { 0: "beginner", 1: "intermediate", 2: "advanced" }
r = itertools.combinations_with_replacement([
    enum["beginner"],
    enum["intermediate"],
    enum["advanced"],
], 2)
r = list(r)

In [6]:
r

[(0, 0), (0, 1), (0, 2), (1, 0), (1, 1), (1, 2), (2, 0), (2, 1), (2, 2)]

In [5]:
x = [enum["beginner"],
    enum["intermediate"],
    enum["advanced"]]


r = [p for p in itertools.product(x, repeat=2)]

r = list(r)

In [7]:
r = [
    (4,8,5,6,7,1),
    (3,8,8,5,1,1),
    (3,6,8,7,1,8),
    (4,1,10,6,1,8),
    (9,5,3,6,5,1),
    (4,10,10,7,8,8),
    (3,10,6,7,1,1),
    (3,7,6,8,5,1),
    (3,10,8,8,6,5)
]

In [12]:
def format_result(i, j, sim, idx = ""):
    return {
        f"i{idx}": i,
        f"j{idx}": j,
#        f"Word{idx}": inverse_enum[i[2]],
        f"Similaridade{idx}": sim
    }

## Continuous Saturated Merit

$$S_{ij} = \dfrac{\sum_{k = 0}^{N-1} \frac{r_{ik} - r_{jk}}{r_{k, max}} \cdot (\frac{r_{jk}}{r_{k, max}} + 1) \cdot c_{ijk}}
           {\sum_{k = 0}^{N-1} {\frac{r_{jk}}{r_{k, max}} + 1}} + 1$$
           
$$S_{ij} \in \mathclose[0,2\mathopen]$$


$$ c_{ijk} = \begin{cases} 1, & \mbox{if } r_{ik} < r_{jk} \\ c_{k}, & \mbox{if } r_{ik} \geq r_{jk} \end{cases} $$

$$ 0 \leq c_k \leq 1 $$
$$ c_{k,min} = 0; c_{k,max} = 1 $$

$$ X =  \text{rank classes}  = \{ \text{"beginner"}, \text{"intermediate"}, \text{"advanced"} \} $$
$$ r_{ik} = X^{-1}(x_{ik}) $$
$$ X^{-1}(\text{"beginner"}) = 0, X^{-1}(\text{"intermediate"}) = 1 \text{ and } X^{-1}(\text{"advanced"}) = 2 $$

In [10]:
def exponential_sim_match(i, j, C = [1]*6, idx = ""):
    numerator = 0
    denominator = 0
    
    for _i, _j, _c in list(zip(i, j, C)):
        _numerator = ((_i/10) - (_j/10))*((_j/10) +1)
        
        if _i > _j:
            numerator += _numerator*_c
        else:
            numerator += _numerator
        
        denominator += ((_j/10) + 1)        
        
    return format_result(i, j, '%f' % (numerator/denominator), idx)

In [8]:
[exponential_sim_match([1, 2], [2, 0]), exponential_sim_match([2, 1], [2, 0])]

[{'i': [1, 2], 'j': [2, 0], 'Similaridade': '-0.166667'},
 {'i': [2, 1], 'j': [2, 0], 'Similaridade': '0.083333'}]

In [9]:
[
    exponential_sim_match([0, 2], [2, 0]),
    exponential_sim_match([0, 2], [2, 1]),
    exponential_sim_match([0, 2], [2, 2])
]

[{'i': [0, 2], 'j': [2, 0], 'Similaridade': '-0.333333'},
 {'i': [0, 2], 'j': [2, 1], 'Similaridade': '-0.357143'},
 {'i': [0, 2], 'j': [2, 2], 'Similaridade': '-0.500000'}]

In [8]:
[
    exponential_sim_match([1, 2], [2, 0]),
    exponential_sim_match([1, 2], [2, 1]),
    exponential_sim_match([1, 2], [2, 2])
]

[{'i': [1, 2], 'j': [2, 0], 'Similaridade': '0.000000'},
 {'i': [1, 2], 'j': [2, 1], 'Similaridade': '-0.071429'},
 {'i': [1, 2], 'j': [2, 2], 'Similaridade': '-0.250000'}]

In [64]:
def cosine_sim_match(i, j, idx=""):
    return format_result(i, j, '%f' % (cosine_similarity([i], [j])), idx)

In [79]:
def manhattan_sim_match(i, j, idx=""):
    sim = 1 / (1 + distance.minkowski(i, j, 1))
    return format_result(i, j, '%f' % sim, idx)

In [66]:
def euclidian_sim_match(i, j, idx=""):
    sim = 1 / (1 + distance.minkowski(i, j, 2))
    return format_result(i, j, '%f' % sim, idx)

In [67]:
def pearson_sim_match(i, j, idx=""):
    sim, _ = pearsonr(i, j)
    return format_result(i, j, '%f' % sim, idx)

# Comparing with other measures

## Cosine

In [ ]:
list(map(lambda x: list(map(lambda i: cosine_sim_match(i, x), r)), r))

## Manhattan

In [ ]:
list(map(lambda x: list(map(lambda i: manhattan_sim_match(i, x), r)), r))

## Euclidian

In [ ]:
list(map(lambda x: list(map(lambda i: euclidian_sim_match(i, x), r)), r))

## Pearson

In [ ]:
list(map(lambda x: list(map(lambda i: pearson_sim_match(i, x), r)), r))

## Our measure

In [18]:
#a = list(map(lambda x: list(map(lambda i: exponential_sim_match(i, x), r)), r))
i = 1
for _r in r:
    print(str(i) + str(exponential_sim_match(_r, [0, 0, 0, 0, 0, 0])))
    i+=1

1{'i': (4, 8, 5, 6, 7, 1), 'j': [0, 0, 0, 0, 0, 0], 'Similaridade': '0.516667'}
2{'i': (3, 8, 8, 5, 1, 1), 'j': [0, 0, 0, 0, 0, 0], 'Similaridade': '0.433333'}
3{'i': (3, 6, 8, 7, 1, 8), 'j': [0, 0, 0, 0, 0, 0], 'Similaridade': '0.550000'}
4{'i': (4, 1, 10, 6, 1, 8), 'j': [0, 0, 0, 0, 0, 0], 'Similaridade': '0.500000'}
5{'i': (9, 5, 3, 6, 5, 1), 'j': [0, 0, 0, 0, 0, 0], 'Similaridade': '0.483333'}
6{'i': (4, 10, 10, 7, 8, 8), 'j': [0, 0, 0, 0, 0, 0], 'Similaridade': '0.783333'}
7{'i': (3, 10, 6, 7, 1, 1), 'j': [0, 0, 0, 0, 0, 0], 'Similaridade': '0.466667'}
8{'i': (3, 7, 6, 8, 5, 1), 'j': [0, 0, 0, 0, 0, 0], 'Similaridade': '0.500000'}
9{'i': (3, 10, 8, 8, 6, 5), 'j': [0, 0, 0, 0, 0, 0], 'Similaridade': '0.666667'}


In [14]:
a

[[{'i': (4, 8, 5, 6, 7, 1),
   'j': (4, 8, 5, 6, 7, 1),
   'Similaridade': '0.000000'},
  {'i': (3, 8, 8, 5, 1, 1),
   'j': (4, 8, 5, 6, 7, 1),
   'Similaridade': '-0.095604'},
  {'i': (3, 6, 8, 7, 1, 8),
   'j': (4, 8, 5, 6, 7, 1),
   'Similaridade': '-0.015385'},
  {'i': (4, 1, 10, 6, 1, 8),
   'j': (4, 8, 5, 6, 7, 1),
   'Similaridade': '-0.083516'},
  {'i': (9, 5, 3, 6, 5, 1),
   'j': (4, 8, 5, 6, 7, 1),
   'Similaridade': '-0.052747'},
  {'i': (4, 10, 10, 7, 8, 8),
   'j': (4, 8, 5, 6, 7, 1),
   'Similaridade': '0.242857'},
  {'i': (3, 10, 6, 7, 1, 1),
   'j': (4, 8, 5, 6, 7, 1),
   'Similaridade': '-0.053846'},
  {'i': (3, 7, 6, 8, 5, 1),
   'j': (4, 8, 5, 6, 7, 1),
   'Similaridade': '-0.020879'},
  {'i': (3, 10, 8, 8, 6, 5),
   'j': (4, 8, 5, 6, 7, 1),
   'Similaridade': '0.138462'}],
 [{'i': (4, 8, 5, 6, 7, 1),
   'j': (3, 8, 8, 5, 1, 1),
   'Similaridade': '0.046512'},
  {'i': (3, 8, 8, 5, 1, 1),
   'j': (3, 8, 8, 5, 1, 1),
   'Similaridade': '0.000000'},
  {'i': (3, 6, 8, 7,

In [46]:
com_can = []
for company in a:
    _r = []
    for candidate in company:
        _r.append(candidate['Similaridade'])
    com_can.append(_r)
    
t = numpy.transpose(com_can)
for candidate in t:
    print(' & '.join(map(lambda o: str(Decimal(str(o)).normalize()), candidate)) + " \\\\")



0 & -0.3 & -0.666667 & -0.3 & -0.5 & -0.785714 & -0.666667 & -0.785714 & -1 \\
0 & 0 & -0.333333 & -0.3 & -0.25 & -0.5 & -0.666667 & -0.571429 & -0.75 \\
0 & 0 & 0 & -0.3 & -0.25 & -0.214286 & -0.666667 & -0.571429 & -0.5 \\
0 & -0.3 & -0.666667 & 0 & -0.25 & -0.571429 & -0.333333 & -0.5 & -0.75 \\
0 & 0 & -0.333333 & 0 & 0 & -0.285714 & -0.333333 & -0.285714 & -0.5 \\
0 & 0 & 0 & 0 & 0 & 0 & -0.333333 & -0.285714 & -0.25 \\
0 & -0.3 & -0.666667 & 0 & -0.25 & -0.571429 & 0 & -0.214286 & -0.5 \\
0 & 0 & -0.333333 & 0 & 0 & -0.285714 & 0 & 0 & -0.25 \\
0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 \\


In [12]:
from decimal import Decimal

In [17]:
[exponential_sim_match(j=[0,0], i= [2,2], C=[1, 1]),
exponential_sim_match(j = [2,0], i= [1,1], C=[1, 0]),
exponential_sim_match(j = [2,0], i= [1,2], C=[1, 0])
                     ]

[{'i': [2, 2], 'j': [0, 0], 'Similaridade': '1.000000'},
 {'i': [1, 1], 'j': [2, 0], 'Similaridade': '-0.500000'},
 {'i': [1, 2], 'j': [2, 0], 'Similaridade': '-0.500000'}]

## Análise

### Mérito Continuo

- Usuário tem a opção de valorizar os conhecimentos a mais de forma gradual
- Usuário deve ter em mente que se a maioria dos atributos não valorizar o conhecimento a mais, as similaridades serão muito similares, indepedente do nível de conhecimento
- O mérito da cada variável é independente
- Valoriza os conhecimentos de forma quadrádica (ex, doutorado >>> mestrado >> bacharel > técnico)


### Mérito Binário

- Não faz sentido permitir méritos contínuos com a estratégia de valorizar os conhecimentos a mais de todos os atributos quando mais do que a metade dos méritos for 0
    - Todos os valores contínuos dos méritos iriam se tornar 1? Os valores diferentes de 0 iriam se manter?
- Usuário deve ter em mente que a valorização dos conhecimentos a mais ocorrerá em todos os atributos se houver mais do que a metade dos atributos de zeros como méritos 

---------------------

Aplicações

- o valor padrão de mérito pode ser 1, tanto para contínuo e binário, sempre valorizar os conhecimentos a mais. Se o usuário quiser um valor diferente para certo atributo, ele que mude
- para vários atributos, definir um valor de mérito contínuo será uma tarefa tediosa, portanto, a melhor opção talvez seria a opção dos méritos binários
- para poucos atributos, não é trabalhoso definir o mérito para cada um, portanto, a melhor opção pode ser a dos méritos contínuos


In [13]:
j_s1 = [2, 2]
c_s1 = [0, 0]
result = list(map(lambda i: exponential_sim_match(i, j_s1, c_s1, 1), list(r)))

j_s2 = [0, 0, 0]
c_s2 = [1, 1, 1]
result2 = list(map(lambda i: exponential_sim_match(i, j_s1, c_s2, 2), list(r)))

In [18]:
[
    exponential_sim_match(i = [2, 2], j = [0, 0], C = [1, 1]),
    exponential_sim_match(i = [1, 1], j = [2, 2], C = [0, 0]),
]

[{'Excel': 2, 'PPT': 2, 'Similaridade': '2.000000'},
 {'Excel': 1, 'PPT': 1, 'Similaridade': '0.500000'}]

In [31]:
result = sorted(result, key=lambda x: x["Similaridade1"])
result2 = sorted(result2, key=lambda x: x["Similaridade2"])

In [14]:
for i, v in enumerate(result): result[i]['idx1'] = i
for i, v in enumerate(result): result2[i]['idx2'] = i

In [15]:
result1_df = pandas.DataFrame(result)
result2_df = pandas.DataFrame(result2)

print(f"Empresa1: {str(j_s1)} | c1 = {str(c_s1)} ======= Empresa2: {str(j_s2)} | c2 = {str(c_s2)} ")
pandas.concat([result1_df, result2_df], axis=1)

Empresa1: [2, 2, 2] | c1 = [1, 1, 0] ======= Empresa2: [0, 0, 0] | c2 = [1, 1, 1] 


,Excel1,PPT1,Word1,Similaridade1,idx1,Excel2,PPT2,Word2,Similaridade2,idx2
0,nada,nada,nada,-0.444444,0,nada,nada,nada,-0.444444,0
1,nada,nada,básico,-0.407407,1,nada,nada,básico,-0.407407,1
2,nada,nada,médio,-0.296296,2,nada,nada,médio,-0.296296,2
3,nada,nada,avançado,-0.296296,3,nada,nada,avançado,-0.111111,3
4,nada,básico,nada,-0.407407,4,nada,básico,nada,-0.407407,4
5,nada,básico,básico,-0.370370,5,nada,básico,básico,-0.370370,5
6,nada,básico,médio,-0.259259,6,nada,básico,médio,-0.259259,6
7,nada,básico,avançado,-0.259259,7,nada,básico,avançado,-0.074074,7
8,nada,médio,nada,-0.296296,8,nada,médio,nada,-0.296296,8
9,nada,médio,básico,-0.259259,9,nada,médio,básico,-0.259259,9
